# Simulations for Clustered Switchback Experiments
<a id='toc'></a>

## Table of Contents 
[Section 1: Model](#sec1)

[Section 2: Step by Step Guide](#step_by_step)

In [46]:
import time, importlib, sys, os, pickle
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib.pyplot as plt

# set up directory
current_dir = os.getcwd() # Get current directory
print("current dir:", current_dir)

parent_dir = os.path.dirname(current_dir) # Go up by one level
sys.path.append(parent_dir) # Add parent dir to sys path

from helpers import utils, graph_helpers, mdp_helpers, stats_helpers, print_nicely

current dir: /Users/cleeyu/Documents/VSCode_projects/clustered_switchback_v20260203


## 1. Model <a id='sec1'></a> 
We will use a linear reward function
$$\mu_{it}(S_{it}): = E[Y_{it} \mid S_{it}, W] = b_{it} + a_{it} \frac {S_{it}} m.$$
We devide by $m$ to normalize. The state evolves as a clipped random walk with states $\{0,1,...,m\}$. Specifially, define *competition level* as the proportion of neighbors assigned arm $1$, formally,  
$$C_{it} = \frac{\sum_{j\in {\cal N}(i)\backslash i}W_{jt}}{|{\cal N}(i)\backslash i|}.$$
The probability of going up conditional on non-lazy event is 
$$P^{\rm up}_{it} = \sigma(\alpha_{it} + \beta_{it}W_{it} - \gamma_{it} C_{it})$$
(expect at the ceiling), where $\sigma$ is the sigmod function.
Accounting for the laziness $L\in [0,1]$, the probability of going up is then 
$$(1-L)\cdot P^{\rm up}_{it}.$$
Assume that $\gamma_{it} \leq \beta_{it}$ such that under full treatment the net effect is still positive.
For a given $\delta$, the exposure mapping is computed as the joint of $W_{it} = a$ AND at least $(1-\delta)$ fraction of its spatiotemporal neighbors excluding $(i,t)$ itself have treatment equal to $a$.

<a id='step_by_step'></a> 

## 2. Step-by-step guide

We now combine the components in the above sections.
Pipeline:
1. Define instance (interference graph, reward func, transition probability)
2. Generate treatment assignment W
3. Run the MDP induced by W
4. estimate ATE using HT estimator

Back to [Table of Contents](#toc)

In [47]:
sim_config = {
    "T": 100, # time horizon
    "num_states": 5, # maximum inventory
    'time_block_length': 10, # time blocks used for randomized design
    "recency": 3, # how many rounds to look back in HT/Hajek estimator 
    "delta": 0.1, #  expo map X_ita^r = 1 if more than 1-delta spatio-temp neighbors are assigned arm a
    'num_W_for_computing_prop': 1000, # num of trtmt asgn mtx used for finding prop score (monte carlo) 
}

## 2.1: Setup interference graph and spatial clusters

In [ ]:
importlib.reload(graph_helpers)
importlib.reload(stats_helpers)

########## STRICT LATTICE GRAPH ##########

sim_config.update({
    "n": 60**2, # if constructing a strict lattice graph, this quantity must be a perfect square
    'num_cells_per_dim': 2, # defines spatial block for randomized design
    "kappa": 2, # two nodes interfere (thru reward and transition) if L1 dist <= kappa 
})

# create interference graph
adj_matrix = graph_helpers.generate_interference_graph_from_lattice(
    sqrt_n=int(np.sqrt(sim_config['n'])), 
    kappa=sim_config['kappa']
    )
print(np.mean(np.sum(adj_matrix,axis=1)))

# setup spatial clusters
cluster_matrix = graph_helpers.generate_clusters_from_lattice(sqrt_n=int(np.sqrt(sim_config['n'])), num_cells_per_dim=sim_config['num_cells_per_dim'])
adj_clusters = np.matmul(adj_matrix,cluster_matrix) 
print(np.mean(np.sum(adj_clusters,axis=1)))

In [48]:
importlib.reload(graph_helpers)
importlib.reload(stats_helpers)

########## SPATIAL GRAPH WITH UNIFORM COORDS ##########

sim_config.update({
    "n": 3600, 
    'num_cells_per_dim': 2, # defines spatial partition of unit square used for randomized design
    "kappa": 0.025, # two nodes interfere (thru reward and transition) if Euclidean dist between coordinates <= kappa 
})

# create interference graph
coords_array = graph_helpers.generate_random_points(sim_config['n'])
adj_matrix = graph_helpers.build_adjacency_matrix_from_coords(coords_array, sim_config['kappa'])
print(np.mean(np.sum(adj_matrix,axis=1)))

# setup spatial clusters
cluster_matrix = graph_helpers.spatial_clustering_map(coords_array, sim_config['num_cells_per_dim'])
adj_clusters = np.matmul(adj_matrix,cluster_matrix) 
print(np.mean(np.sum(adj_clusters,axis=1)))

7.993888888888889
7.993888888888889


## 2.2: Initialize MC and find true ATE using Monte Carlo 
Back to [Table of Contents](#toc)


In [49]:
importlib.reload(mdp_helpers)

# initialize Markov chain model

MC = mdp_helpers.InventoryMarkovChain(
    max_inventory=sim_config['num_states'],
    adj_matrix=adj_matrix,
    num_rounds=sim_config['T'],
    C_lazy = 0.1,
    C_alpha = -5,
    C_beta = 10,
    C_gamma = 0,
    C_baseline = 100,
    C_slope= 10)

In [51]:
# approximate ATE via Monte Carlo

start_time = time.time()

num_sims_apx_ate = 1000
initial_state = 2

sim_results_0 = MC.simulate_MC(initial_state * np.ones((sim_config['n'])), np.zeros((sim_config['n'], sim_config['T'],num_sims_apx_ate)), use_sigmoid=True)
sim_results_1 = MC.simulate_MC(initial_state * np.ones((sim_config['n'])), np.ones((sim_config['n'], sim_config['T'],num_sims_apx_ate)), use_sigmoid=True)

all_0_mean = np.mean(sim_results_0["rewards"])
all_1_mean = np.mean(sim_results_1["rewards"])

print("="*20 + " True ATE (apx'ed using Monte Carlo) " + "="*20)
print(f"Mean reward under all-1 vs. all-0: {all_1_mean:.4f} vs. {all_0_mean:.4f}  ")
true_ATE = all_1_mean - all_0_mean
print(f"True ATE: {true_ATE:.4f}")

==================== True ATE (apx'ed using Monte Carlo) ====================
Mean reward under all-1 vs. all-0: 109.9113 vs. 100.0411  
True ATE: 9.8702


## 2.3: Compute propensity scores
Back to [Table of Contents](#toc)

In [ ]:
importlib.reload(stats_helpers)

time_cluster_matrix = stats_helpers.generate_time_blocks(T=sim_config['T'], time_block_length=sim_config['time_block_length'])
time_adj_matrix = np.tril(np.ones((sim_config['T'],sim_config['T'])), k=0) - np.tril(np.ones((sim_config['T'],sim_config['T'])), k=-(sim_config['recency'] + 1))
adj_time_blocks = np.matmul(time_adj_matrix,time_cluster_matrix)

print(np.mean(np.sum(time_adj_matrix,axis=1)))

# Compute propensity score (Monte Carlo)

arms_tensor = stats_helpers.generate_cluster_treatments(cluster_matrix,time_cluster_matrix,num_W=sim_config['num_W_for_computing_prop'])
print(arms_tensor.shape)

emp_prop_score_results = stats_helpers.empirical_propensity_scores(arms_tensor,adj_matrix,time_adj_matrix,sim_config['delta'])
propensity_1_array, propensity_0_array = emp_prop_score_results['propensity_1'], emp_prop_score_results['propensity_0']

print(f"minimum of emp prop score: {np.amin(propensity_1_array)}, {np.amin(propensity_0_array)}")
print(f"total number of zeros: {np.count_nonzero(propensity_1_array == 0)}, {np.count_nonzero(propensity_0_array == 0)}")

# Display emp_prop_score_results
prop_1_mean, prop_0_mean = propensity_1_array.mean(), propensity_0_array.mean()
print(f"mean emp prop score (interior units): {prop_1_mean:.4f}, {prop_0_mean:.4f}") 
n, T = sim_config['n'], sim_config['T']
print(f"expected #(i,t) with X_it=1 is {n*T*prop_1_mean:.2f}, {n*T*prop_0_mean:.2f}")
print(f"%nz in emp prop_score_0, prop_score_1: {len(np.nonzero(propensity_0_array)[0])/(n*T)*100:.2f}%, {len(np.nonzero(propensity_1_array)[0])/(n*T)*100:.2f}%")
utils.print_time()

HT_weights_0 = np.zeros((np.shape(propensity_0_array)))
np.divide(1, propensity_0_array, out = HT_weights_0, where=propensity_0_array != 0)
print(np.mean(HT_weights_0))
print(np.std(HT_weights_0))
print(np.amax(HT_weights_0))
print(np.amin(HT_weights_0))

HT_weights_1 = np.zeros((np.shape(propensity_1_array)))
np.divide(1, propensity_1_array, out = HT_weights_1, where=propensity_1_array != 0)
print(np.mean(HT_weights_1))
print(np.std(HT_weights_1))
print(np.amax(HT_weights_1))
print(np.amin(HT_weights_1))

## 2.4: Simulate experiment and compute HT and Hajek estimators
Back to [Table of Contents](#toc)

In [ ]:
importlib.reload(stats_helpers)

num_iter_sim = 1000

initial_state = 2

start_time = time.time()

W = stats_helpers.generate_cluster_treatments(cluster_matrix,time_cluster_matrix,num_iter_sim)
exposure_results = stats_helpers.exposure_mapping(W, adj_matrix, time_adj_matrix, sim_config['delta'])

sim_results = MC.simulate_MC(initial_state * np.ones((sim_config['n'])), W, use_sigmoid=True)
rewards = sim_results["rewards"]

rewards_0 = rewards[W == 0]
rewards_1 = rewards[W == 1]
print(np.mean(rewards_0))
print(np.std(rewards_0))
print(np.amax(rewards_0))
print(np.amin(rewards_0))
print(np.mean(rewards_1))
print(np.std(rewards_1))
print(np.amax(rewards_1))
print(np.amin(rewards_1))



In [ ]:
ht_results = stats_helpers.horvitz_thompson(rewards,exposure_results['exposure_1'],exposure_results['exposure_0'],propensity_1_array,propensity_0_array)
ate_estimate_ht = ht_results['ate_estimate_ht']

print("\n" + "="*20 + " Horvitz-Thompson Estimates " + "="*20)
mean_HT_est, var_HT_est = ate_estimate_ht.mean(), ate_estimate_ht.var()
print(f"mean_HT_est: {mean_HT_est:.4f}\nbias: {mean_HT_est - true_ATE:.4f}\n" + f"var_HT_est: {var_HT_est:.4f}")

In [ ]:
hajek_results = stats_helpers.hajek(rewards,exposure_results['exposure_1'],exposure_results['exposure_0'],propensity_1_array,propensity_0_array)
ate_estimate_hajek = hajek_results['ate_estimate_hajek']

print("\n" + "="*20 + " Hajek Estimates " + "="*20)
mean_Hajek_est, var_Hajek_est = ate_estimate_hajek.mean(), ate_estimate_hajek.var()
print(f"mean_HT_est: {mean_Hajek_est:.4f}\nbias: {mean_Hajek_est - true_ATE:.4f}\n" + f"var_Hajek_est: {var_Hajek_est:.4f}")